In [3]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 9.6 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install pandas 
!pip install psycopg2-binary


[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install sqlalchemy

  Using cached SQLAlchemy-2.0.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [ ]:
import logging
from kafka import KafkaConsumer
import requests
import json
import psycopg2
import uuid
import pandas as pd
import time

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

resp = []

# Replace with your Kafka broker address(es)
brokers = [f"{host}:9092"]

# Replace with your topic name
topic = "three"


db_details = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

try:
    conn = psycopg2.connect(**db_details)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()
    
try:
    fetch_query = "SELECT * FROM malware_model_metrics where in_use is true LIMIT 1;"
    model = pd.read_sql(fetch_query, conn)
except Exception as e:
    print(f"Failed to fetch data: {e}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

# KServe inference service URL
kserve_url = f"http://{model['name'][0]}-version{model['version'][0]}-md.kubeflow-user-example-com.svc.cluster.local/v1/models/{model['name'][0]}-version{model['version'][0]}-md:predict"

# Create a Kafka consumer
logger.info("Creating Kafka consumer...")
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=brokers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda v: v.decode('utf-8')
)

logger.info(f"Subscribed to topic: {topic}")


# Function to send data to KServe
def send_to_kserve(data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(kserve_url, headers=headers, data=json.dumps(data))
    return response.json()

def outcome_to_database(values):
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    # Data to insert
    outcome_data = {
        'uid': values[0],  # Generating a unique ID
        'outcome': values[1]  # This can be 0 or 1
    }

    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()

    # Insert data into the table
    try:
        insert_query = """
        INSERT INTO malware_outcomes (uid, outcome)
        VALUES (%s, %s)
        """
        cursor.execute(insert_query, (outcome_data['uid'], outcome_data['outcome']))
 
        update_query = """
        UPDATE malware_data
        SET outcome = %s
        WHERE uid = %s
        """
        cursor.execute(update_query, (outcome_data['outcome'], outcome_data['uid']))
        conn.commit()
        print("Data inserted and updated successfully.")
    except Exception as e:
        print(f"Failed to insert/update data: {e}")
        conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("PostgreSQL connection closed.")

        
        

try:
    for message in consumer:
        try:
            resp = []
            message = json.loads(message.value)  # Message value is already a dict
            print(f"Received message: {message}")
            uid = message.pop('uid', None)
            resp.append(uid)
            #print(f"new message: {message}")
            # Prepare data for KServe
            kserve_payload = {
                "instances": [list(message.values())]  # Reshape the data to 2D array
            }

            max_retries = 5
            for attempt in range(max_retries):
                try:
                    # Send data to KServe
                    kserve_response = send_to_kserve(kserve_payload)
                    if kserve_response:
                        resp.append(kserve_response['predictions'][0])
                        print(f"KServe response: {kserve_response}")
                        print(resp)
                        outcome_to_database(resp)
                        break 
                    else:
                        raise ValueError("No response from KServe")
                except Exception as e:
                    if attempt < max_retries - 1:
                        print(f"Attempt {attempt + 1} failed: {e}. Retrying in 5 seconds...")
                        time.sleep(5)
                    else:
                        logger.error(f"Failed after {max_retries} attempts: {e}")
                        raise
        except json.JSONDecodeError:
            print(f"Received non-JSON message: {message.value}")
        except Exception as e:
            logger.error(f"Error sending to KServe: {e}")
        #logger.info(f"Partition: {message.partition}, Offset: {message.offset}")
except KeyboardInterrupt:
    logger.info("Consumer stopped manually.")
finally:
    consumer.close()
    logger.info("Consumer closed.")


Connected to PostgreSQL successfully.


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
INFO:__main__:Creating Kafka consumer...
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.90.179.164:9092 <connecting> [IPv4 ('54.90.179.164', 9092)]>: connecting to 54.90.179.164:9092 [('54.90.179.164', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.90.179.164:9092 <connecting> [IPv4 ('54.90.179.164', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('three',)
INFO:__main__:Subscribed to topic: three
INFO:kafka.cluster:Gr

Received message: {'SizeOfOptionalHeader': -0.36050012266580916, 'MinorLinkerVersion': -0.37627767498141335, 'SizeOfCode': 0.7319599292637426, 'SizeOfInitializedData': -0.21890950816598462, 'SizeOfUninitializedData': -0.12753906510031895, 'AddressOfEntryPoint': -0.5341880246042172, 'BaseOfCode': -0.10815588643796789, 'ImageBase': -0.014265727914658645, 'SectionAlignment': -0.027858329654181283, 'MajorOperatingSystemVersion': -3.783148235437173, 'MinorOperatingSystemVersion': -0.815858751009834, 'MinorImageVersion': -0.08601091612828965, 'MajorSubsystemVersion': -8.145060131390686, 'SizeOfHeaders': -0.19993366962860798, 'CheckSum': -0.30014209492084976, 'Subsystem': 18.931377785748012, 'DllCharacteristics': -1.1838829445356291, 'SizeOfStackReserve': 0.3632015553566146, 'SizeOfStackCommit': -0.18911470200141214, 'SizeOfHeapReserve': 0.1275466136330668, 'SectionsNb': 1.685718064005637, 'SectionsMeanEntropy': 0.9689674506872942, 'SectionsMeanRawsize': -0.08981867951245306, 'SectionsMeanVir

INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-54-90-179-164.compute-1.amazonaws.com:9094 <connecting> [IPv4 ('54.90.179.164', 9094)]>: connecting to ec2-54-90-179-164.compute-1.amazonaws.com:9094 [('54.90.179.164', 9094) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=2 host=ec2-54-90-179-164.compute-1.amazonaws.com:9094 <connecting> [IPv4 ('54.90.179.164', 9094)]>: Connection complete.
